In [1]:
from astropy.table import Table, Column, table, vstack
from pathlib import Path

import numpy as np
import astropy.units as u

from matplotlib import pyplot as plt

In [2]:
path_raw='../Datos/Raw/'
path_phot_ned = '../Datos/Work/Phot_Tables_NED/' #In this path wi will save the photometry tables for each object
path_phot_cds = '../Datos/Work/Phot_Tables_CDS/'
path_SED = '../Datos/Work/SEDs/'
path_CIGALE_phot = '../Datos/Work/CIGALE_InputPhot/'
path_CIGALE_out = '../Datos/Work/CIGALE_Output/'
path_FINAL = '../Datos/Final/'

In [3]:
date_sample = 'Sep_02_2022'
NeII_NeV_Sy = Table.read(path_raw+'Obj_Sy_Ne-IR_'+date_sample+'.vot',format='votable')

In [4]:
to_rem = [] #Rows to be deleted or removed in out_1
for i in range(len(NeII_NeV_Sy)): 
    if (NeII_NeV_Sy['NED_Ne_IR_info'][i] != 'NeII|NeIII|NeV' and NeII_NeV_Sy['NED_Ne_IR_info'][i]!= 'NeII|NeV'):
        to_rem.append(i)
    
NeII_NeV_Sy.remove_rows(to_rem)
#NeII_NeV_Sy.show_in_notebook()
len(NeII_NeV_Sy)

183

In [5]:
def Neon_info(NED_table, Freq):
    
    '''
    This function create a new table from the Phot NED table
    but with the flux of [NeII] and [NeV]
    '''
    
    NED_table.remove_rows(np.where(NED_table['Flux_Density'].mask)[0])
    NED_table.remove_rows(np.where(NED_table['NED_Units'] == 'Jy')[0])
    NED_table.remove_rows(np.where(NED_table['Observed_Passband'] == '[Cl II] line (IRS)')[0])
    NED_table.remove_rows(np.where(NED_table['Observed_Passband'] == '[Cl II] 14.4 (IRS)')[0])
    NED_table.remove_rows(np.where(NED_table['Observed_Passband'] == '[Cl II] (IRS)')[0])
    NED_table.remove_rows(np.where(NED_table['Observed_Passband'] == '[Cl II] 14.37 (IRS)')[0])
    
    Ne_idx = [np.where(NED_table['Frequency'] == freq)[0] for freq in Freq]
    #print(Ne_idx)
    #print(len(Ne_idx))
    i = []
    Ne_info = Table()
    for idx in range(len(Freq)):
        Ne_info = vstack([Ne_info,NED_table[Ne_idx[idx]]])
    
    for ind in range(len(Ne_info)):
        i.append(ind)
    
    Ne_info['idx'] = i
    Ne_inf = Ne_info['idx','Observed_Passband','Flux_Density','NED_Uncertainty','NED_Units','Refcode']
    return Ne_inf

In the galaxies where there are more than one data of [NeII] or [NeV], we remove the rows if per bibcode there is data for only one the them. The idea is to have both [NeII] and [NeV] data per bibcode:

In [6]:
passband = Table()
refcode = Table()


not_same_bib = 0

for id in range(len(NeII_NeV_Sy)):
    NED_phot = Table()
    obj = NeII_NeV_Sy['Main_Id'][id]
    print(id)
    print('Galaxy: ',obj)

    NED_phot = Table.read(path_phot_ned+obj+'_NED_phot_tables'+'.vot', format='votable')
    print('Phot table lenght:',len(NED_phot))
   
    
    NeII_NeV_Freq = [20900000000000.0,21000000000000.0,23400000000000.0]
    NeII_NeV_inf = Neon_info(NED_phot, NeII_NeV_Freq)
    bibcod, count = np.unique(NeII_NeV_inf['Refcode'], return_counts=True)
    #print(len(NeII_NeV_inf))

    
    if len(NeII_NeV_inf)>2:
        print(NeII_NeV_inf)
        for idx,bib in enumerate(bibcod):
            i = np.where(NeII_NeV_inf['Refcode'] == bib)[0]
            #The ratio [NeV]/[NeII] is calculated for only the bibcodes that have both flux data
            if len(i)>=2:
                #print(len(i))
                NeV_i = 'nan'
                NeII_i = 'nan'
                for j in range(len(i)):
                    if '[Ne V]' in NeII_NeV_inf['Observed_Passband'][i[j]]:
                        NeV_i = i[j]
                    elif '[Ne II]' in NeII_NeV_inf['Observed_Passband'][i[j]]:
                        NeII_i = i[j]
                #If of both data per bibcode, one of [Ne]lines is misssing then the row is removed
                if NeV_i == 'nan' or NeII_i == 'nan':
                    NeII_NeV_inf.remove_rows(np.where(NeII_NeV_inf['Refcode'] == bib)[0])
                else:
                    NeV = NeII_NeV_inf['Flux_Density'][NeV_i]
                    NeII = NeII_NeV_inf['Flux_Density'][NeII_i]
                    NeV_NeII_ratio = NeV/NeII
                    print('For bibcode',bib,', [Ne V] index is:',NeV_i,'and [Ne II] index is:',NeII_i)
                    print('For bibcode', bib,'[NeV]/[NeII] ratio is:',NeV_NeII_ratio)
            #If there is only one data flux per bibcode then the row is removed
            elif len(i)<2:
                NeII_NeV_inf.remove_rows(np.where(NeII_NeV_inf['Refcode'] == bib)[0])               
    elif len(NeII_NeV_inf)==2:
        NeV = NeII_NeV_inf['Flux_Density'][0]
        NeII = NeII_NeV_inf['Flux_Density'][1]
        NeV_NeII_ratio = NeV/NeII
        if NeII_NeV_inf['Refcode'][0] == NeII_NeV_inf['Refcode'][1]:
            print('For bibcode',bib,', [Ne V] index is: 0 and [Ne II] index is: 1')
            print('For bibcode', bib,'[NeV]/[NeII] ratio is:',NeV_NeII_ratio)
            print(NeII_NeV_inf,'\n')
        elif NeII_NeV_inf['Refcode'][0] != NeII_NeV_inf['Refcode'][1]:
            not_same_bib += 1
                
    passband = vstack([passband,NeII_NeV_inf['Observed_Passband']])
    refcode = vstack([refcode,NeII_NeV_inf['Refcode']])
    print('\n')
    

0
Galaxy:  3C 234.0
Phot table lenght: 253
idx  Observed_Passband   Flux_Density ... NED_Units       Refcode      
                              Jy      ...                              
--- -------------------- ------------ ... --------- -------------------
  0   [Ne V] 14.32 (IRS) 2350000000.0 ...     Jy-Hz 2010ApJ...709.1257T
  1  [Ne V] 14.3 microns 3300000000.0 ...     Jy-Hz 2006ApJ...647..161O
  2   [Ne II] 12.8 (IRS) 1400000000.0 ...     Jy-Hz 2009ApJ...697..182T
  3 [Ne II] 12.81 micron  800000000.0 ...     Jy-Hz 2006ApJ...647..161O
For bibcode 2006ApJ...647..161O , [Ne V] index is: 1 and [Ne II] index is: 3
For bibcode 2006ApJ...647..161O [NeV]/[NeII] ratio is: 4.125


1
Galaxy:  NGC   262
Phot table lenght: 334
idx  Observed_Passband   Flux_Density ... NED_Units       Refcode      
                              Jy      ...                              
--- ------------------- ------------- ... --------- -------------------
  0  [Ne V] 14.32 (IRS)  5820000000.0 ...     Jy-Hz 2

For bibcode 2011ApJS..195...17W , [Ne V] index is: 0 and [Ne II] index is: 1
For bibcode 2011ApJS..195...17W [NeV]/[NeII] ratio is: 0.15295055821371611
idx  Observed_Passband  Flux_Density ... NED_Units       Refcode      
                             Jy      ...                              
--- ------------------- ------------ ... --------- -------------------
  0  [Ne V] 14.32 (IRS)  959000000.0 ...     Jy-Hz 2011ApJ...740...94D
  1 [Ne II] 12.81 (IRS) 6270000000.0 ...     Jy-Hz 2011ApJ...740...94D 



11
Galaxy:  NGC  4941
Phot table lenght: 198
idx  Observed_Passband    Flux_Density ... NED_Units       Refcode      
                               Jy      ...                              
--- -------------------- ------------- ... --------- -------------------
  0   [Ne V] 14.32 (IRS)  8210000000.0 ...     Jy-Hz 2010ApJ...709.1257T
  1 [Ne V] 14.32 Spitzer  9000000000.0 ...     Jy-Hz 2007ApJ...671..124D
  2  [Ne V] 14.3 Spitzer  7200000000.0 ...     Jy-Hz 2010ApJ...725.2270P
  3   

idx  Observed_Passband    Flux_Density ... NED_Units       Refcode      
                               Jy      ...                              
--- -------------------- ------------- ... --------- -------------------
  0 [Ne V] 14.32 Spitzer  9000000000.0 ...     Jy-Hz 2007ApJ...671..124D
  1  [Ne V] 14.3 Spitzer  9000000000.0 ...     Jy-Hz 2010ApJ...725.2270P
  2         [Ne V] 14.32 11200000000.0 ...     Jy-Hz 2011ApJS..195...17W
  3        [Ne II] 12.81 45200000000.0 ...     Jy-Hz 2011ApJS..195...17W
  4 [Ne II] 12.8 Spitzer 42000000000.0 ...     Jy-Hz 2010ApJ...725.2270P
  5    [Ne II] (Spitzer) 39000000000.0 ...     Jy-Hz 2007ApJ...671..124D
For bibcode 2007ApJ...671..124D , [Ne V] index is: 0 and [Ne II] index is: 5
For bibcode 2007ApJ...671..124D [NeV]/[NeII] ratio is: 0.23076923076923078
For bibcode 2010ApJ...725.2270P , [Ne V] index is: 1 and [Ne II] index is: 4
For bibcode 2010ApJ...725.2270P [NeV]/[NeII] ratio is: 0.21428571428571427
For bibcode 2011ApJS..195...17W , [Ne V

idx Observed_Passband  Flux_Density ... NED_Units       Refcode      
                            Jy      ...                              
--- ------------------ ------------ ... --------- -------------------
  0       [Ne V] 14.32 2210000000.0 ...     Jy-Hz 2011ApJS..195...17W
  1      [Ne II] 12.81 3230000000.0 ...     Jy-Hz 2011ApJS..195...17W
  2  [Ne II] (Spitzer) 2000000000.0 ...     Jy-Hz 2007ApJ...671..124D
  3 [Ne II] 12.8 (IRS) 7800000000.0 ...     Jy-Hz 2009ApJ...697..182T
For bibcode 2011ApJS..195...17W , [Ne V] index is: 0 and [Ne II] index is: 1
For bibcode 2011ApJS..195...17W [NeV]/[NeII] ratio is: 0.6842105263157895


34
Galaxy:  Mrk 1513
Phot table lenght: 216
idx  Observed_Passband   Flux_Density ... NED_Units       Refcode      
                              Jy      ...                              
--- -------------------- ------------ ... --------- -------------------
  0    [Ne V] 14.3 (IRS) 3800000000.0 ...     Jy-Hz 2009ApJ...690.1105K
  1 [Ne V] 14.32 Spitzer 

idx  Observed_Passband   Flux_Density ... NED_Units       Refcode      
                              Jy      ...                              
--- ------------------- ------------- ... --------- -------------------
  0  [Ne V] 14.32 (IRS)  1520000000.0 ...     Jy-Hz 2010ApJ...716.1151W
  1        [Ne V] 14.32  1910000000.0 ...     Jy-Hz 2011ApJS..195...17W
  2   [Ne II] (Spitzer) 11000000000.0 ...     Jy-Hz 2007ApJ...671..124D
  3       [Ne II] 12.81 12800000000.0 ...     Jy-Hz 2011ApJS..195...17W
  4 [Ne II] 12.81 (IRS) 10400000000.0 ...     Jy-Hz 2010ApJ...716.1151W
For bibcode 2010ApJ...716.1151W , [Ne V] index is: 0 and [Ne II] index is: 3
For bibcode 2010ApJ...716.1151W [NeV]/[NeII] ratio is: 0.14615384615384616
For bibcode 2011ApJS..195...17W , [Ne V] index is: 1 and [Ne II] index is: 2
For bibcode 2011ApJS..195...17W [NeV]/[NeII] ratio is: 0.14921875


45
Galaxy:  ESO 383-35
Phot table lenght: 271
idx  Observed_Passband  Flux_Density ... NED_Units       Refcode      
          

...


128
Galaxy:  NGC  4593
Phot table lenght: 289
idx  Observed_Passband   Flux_Density ... NED_Units       Refcode      
                              Jy      ...                              
--- -------------------- ------------ ... --------- -------------------
  0         [Ne V] 14.32 3090000000.0 ...     Jy-Hz 2011ApJS..195...17W
  1 [Ne V] 14.32 Spitzer 3480000000.0 ...     Jy-Hz 2008ApJ...674L...9D
  2   [Ne V] 14.32 (IRS) 5560000000.0 ...     Jy-Hz 2010ApJ...716.1151W
  3   [Ne V] 14.32 (IRS) 3090000000.0 ...     Jy-Hz 2010ApJ...709.1257T
  4  [Ne V] 14.3 Spitzer 3500000000.0 ...     Jy-Hz 2010ApJ...725.2270P
  5  [Ne II] 12.81 (IRS) 8310000000.0 ...     Jy-Hz 2010ApJ...716.1151W
  6 [Ne II] 12.8 Spitzer 6800000000.0 ...     Jy-Hz 2010ApJ...725.2270P
  7  [Ne II] 12.81 (IRS) 7340000000.0 ...     Jy-Hz 2010ApJ...709.1257T
  8        [Ne II] 12.81 7340000000.0 ...     Jy-Hz 2011ApJS..195...17W
  9   [Ne II] 12.8 (IRS) 6800000000.0 ...     Jy-Hz 2009ApJ...697..182T
For bibcode 2010A

idx  Observed_Passband   Flux_Density ... NED_Units       Refcode      
                              Jy      ...                              
--- ------------------- ------------- ... --------- -------------------
  0  [Ne V] 14.32 (IRS)   470000000.0 ...     Jy-Hz 2010ApJ...716.1151W
  1  [Ne II] 12.8 (IRS) 19000000000.0 ...     Jy-Hz 2009ApJ...697..182T
  2   [Ne II] (Spitzer) 11000000000.0 ...     Jy-Hz 2007ApJ...671..124D
  3       [Ne II] 12.81 11000000000.0 ...     Jy-Hz 2011ApJS..195...17W
  4 [Ne II] 12.81 (IRS)  9320000000.0 ...     Jy-Hz 2010ApJ...716.1151W
For bibcode 2010ApJ...716.1151W , [Ne V] index is: 0 and [Ne II] index is: 2
For bibcode 2010ApJ...716.1151W [NeV]/[NeII] ratio is: 0.05042918454935622


139
Galaxy:  UGC 12138
Phot table lenght: 120
For bibcode 2011ApJS..195...17W , [Ne V] index is: 0 and [Ne II] index is: 1
For bibcode 2011ApJS..195...17W [NeV]/[NeII] ratio is: 2.0
idx  Observed_Passband   Flux_Density ... NED_Units       Refcode      
                

Phot table lenght: 181
For bibcode 2011ApJS..195...17W , [Ne V] index is: 0 and [Ne II] index is: 1
For bibcode 2011ApJS..195...17W [NeV]/[NeII] ratio is: 0.5774378585086042
idx  Observed_Passband  Flux_Density ... NED_Units       Refcode      
                             Jy      ...                              
--- ------------------- ------------ ... --------- -------------------
  0  [Ne V] 14.32 (IRS) 3020000000.0 ...     Jy-Hz 2011ApJ...740...94D
  1 [Ne II] 12.81 (IRS) 5230000000.0 ...     Jy-Hz 2011ApJ...740...94D 



151
Galaxy:  UGC  3601
Phot table lenght: 69
For bibcode 2011ApJS..195...17W , [Ne V] index is: 0 and [Ne II] index is: 1
For bibcode 2011ApJS..195...17W [NeV]/[NeII] ratio is: 0.3821292775665399
idx  Observed_Passband  Flux_Density ... NED_Units       Refcode      
                             Jy      ...                              
--- ------------------- ------------ ... --------- -------------------
  0  [Ne V] 14.32 (IRS) 2010000000.0 ...     Jy-Hz 2010ApJ

idx  Observed_Passband  Flux_Density ... NED_Units       Refcode      
                             Jy      ...                              
--- ------------------- ------------ ... --------- -------------------
  0  [Ne V] 14.32 (IRS) 6630000000.0 ...     Jy-Hz 2010ApJ...709.1257T
  1        [Ne V] 14.32 6630000000.0 ...     Jy-Hz 2011ApJS..195...17W
  2 [Ne II] 12.81 (IRS) 8520000000.0 ...     Jy-Hz 2010ApJ...709.1257T
  3       [Ne II] 12.81 8520000000.0 ...     Jy-Hz 2011ApJS..195...17W
  4   [Ne II] (Spitzer) 5000000000.0 ...     Jy-Hz 2007ApJ...671..124D
For bibcode 2010ApJ...709.1257T , [Ne V] index is: 0 and [Ne II] index is: 2
For bibcode 2010ApJ...709.1257T [NeV]/[NeII] ratio is: 0.778169014084507
For bibcode 2011ApJS..195...17W , [Ne V] index is: 1 and [Ne II] index is: 3
For bibcode 2011ApJS..195...17W [NeV]/[NeII] ratio is: 0.778169014084507


166
Galaxy:  NGC  4922
Phot table lenght: 87
idx  Observed_Passband    Flux_Density ... NED_Units       Refcode      
            

For bibcode 2011ApJ...740...94D , [Ne V] index is: 0 and [Ne II] index is: 1
For bibcode 2011ApJ...740...94D [NeV]/[NeII] ratio is: 0.5636942675159236
idx Observed_Passband Flux_Density NED_Uncertainty NED_Units       Refcode      
                           Jy                                                   
--- ----------------- ------------ --------------- --------- -------------------
  0      [Ne V] (IRS)  531000000.0                     Jy-Hz 2006ApJ...649...79S
  1     [Ne II] (IRS)  942000000.0                     Jy-Hz 2006ApJ...649...79S 



177
Galaxy:  NGC  6860
Phot table lenght: 140
idx  Observed_Passband  Flux_Density ... NED_Units       Refcode      
                             Jy      ...                              
--- ------------------- ------------ ... --------- -------------------
  0  [Ne V] 14.32 (IRS) 2850000000.0 ...     Jy-Hz 2010ApJ...709.1257T
  1        [Ne V] 14.32 2850000000.0 ...     Jy-Hz 2011ApJS..195...17W
  2  [Ne V] 14.32 (IRS) 2950000000.0 ..

In [7]:
passbands, countspb = np.unique(passband, return_counts=True)
for pb in range(len(passbands)): print(passbands[pb][0])

[Ne II] (IRS)
[Ne II] (Spitzer)
[Ne II] 12.8 (IRS)
[Ne II] 12.8 Spitzer
[Ne II] 12.81
[Ne II] 12.81 (IRS)
[Ne II] 12.81 micron
[Ne II] line (IRS)
[Ne V] (IRS)
[Ne V] 14.3 (IRS)
[Ne V] 14.3 Spitzer
[Ne V] 14.3 microns
[Ne V] 14.32
[Ne V] 14.32 (IRS)
[Ne V] 14.32 Spitzer
[Ne V] line (IRS)


In [8]:
refcodes, countsrc = np.unique(refcode, return_counts=True)
for rc in range(len(refcodes)): print(refcodes[rc][0])

2004ApJS..154..178A
2006ApJ...640..204A
2006ApJ...647..161O
2006ApJ...649...79S
2007ApJ...656..148A
2007ApJ...660..117C
2007ApJ...671..124D
2009ApJ...690.1105K
2009MNRAS.398.1165G
2010ApJ...709.1257T
2010ApJ...716.1151W
2010ApJ...725.2270P
2011A&A...533A..56P
2011ApJ...730...28P
2011ApJ...740...94D
2011ApJ...740...99D
2011ApJS..193...18W
2011ApJS..195...17W
2012ApJ...744....2A
2012ApJ...747...46P
2012ApJ...747...95G


In [9]:
print('Number of galaxies with diferent bibcode for [NeV] and [NeII] fluxes:',not_same_bib)

Number of galaxies with diferent bibcode for [NeV] and [NeII] fluxes: 4
